# North Carolina Ensemble Analysis

In this notebook I generate an ensumble using the modified Metropolis Hastings algorithm specified in the paper and also generate a traditional ensemble using normal constraints.

In [1]:
# Python tools for output, directory management, and graph management 
import csv
import os
from functools import partial
import json
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from networkx import is_connected, connected_components
import requests
import zipfile
import io

#gerry chain tools 
from gerrychain import (
    Election,
    Graph,
    MarkovChain,
    Partition,
    accept,
    constraints,
    updaters,
    GeographicPartition,
)
from gerrychain.metrics import efficiency_gap, mean_median
from gerrychain.proposals import recom, propose_random_flip
from gerrychain.updaters import cut_edges, Tally
from gerrychain.constraints import single_flip_contiguous
from gerrychain.accept import always_accept

In [2]:
def extract_github_data(url):
    r = requests.get(url, stream=True)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall("data")

In [3]:
url = "https://raw.github.com/mggg-states/NC-shapefiles/master/NC_VTD.zip"
extract_github_data(url)
df = gpd.read_file("./Data/NC_VTD.shp")
df.set_index("VTD", inplace=True)
df['index'] = df.index

In [4]:
#graph = Graph.from_geodataframe(df)

In [5]:
problems = ['3702144.1', '3705722', '37163LAKE', '37179041', '37193108']

In [6]:
for problem in problems:
    df.loc[problem, "geometry"] = df.loc[problem, "geometry"].buffer(0)

In [21]:
graph = Graph.from_geodataframe(df, cols_to_add='VTD_Key')

TypeError: from_geodataframe() got an unexpected keyword argument 'cols_to_add'

In [9]:
elections = [
    Election("EL16G_US_3", {"Democratic": "EL16G_US_1", "Republican": "EL16G_USS_"}),
    Election("EL16G_PR_T", {"Democratic": "EL16G_PR_D", "Republican": "EL16G_PR_R"}),
    Election("EL12G_PR_T", {"Democratic": "EL12G_PR_D", "Republican": "EL12G_PR_R"}),
    Election("EL14G_US_4", {"Democratic": "EL14G_US_1", "Republican": "EL14G_USS_"})
]

In [10]:
my_updaters = {"population": updaters.Tally("PL10AA_TOT", alias="population")}

In [11]:
election_updaters = {election.name: election for election in elections}
my_updaters.update(election_updaters)

In [12]:
initial_partition = GeographicPartition(graph, assignment="judge", updaters=my_updaters)

KeyError: 'judge'

In [ ]:
for node in graph.nodes():
    print(node)

In [13]:
graph.nodes['3700103N']

{'boundary_node': False, 'area': 5949265.820130458}

In [ ]:
df

In [14]:
df['VTD']{['3700103N}

,ALAND10,AWATER10,County,VTD_Key,VTD_Name,PL10AA_TOT,PL10VA_TOT,EL08G_GV_D,EL08G_GV_R,EL08G_GV_L,...,HVAP,WVAP,BVAP,AMINVAP,ASIANVAP,NHPIVAP,OTHERVAP,2MOREVAP,geometry,index
VTD,,,,,,,,,,,,,,,,,,,,,
3700106W,1951716.0,32157.0,37001,3700106W,06W,1973,1505,330,301,24,...,211,899,338,4,32,0,0,21,"POLYGON ((572318.656 259054.912, 572364.420 25...",3700106W
3700112E,2887286.0,0.0,37001,3700112E,12E,3391,2503,586,263,27,...,622,1085,712,19,38,0,2,25,"POLYGON ((572318.656 259054.912, 572310.834 25...",3700112E
3700112W,2631987.0,0.0,37001,3700112W,12W,2744,2156,646,671,46,...,121,1740,243,6,16,5,2,23,"POLYGON ((569322.877 259376.484, 569315.197 25...",3700112W
3700106N,4284967.0,10517.0,37001,3700106N,06N,4468,3167,885,442,39,...,623,1450,1027,18,29,0,2,18,"POLYGON ((574366.254 259452.385, 574414.625 25...",3700106N
37001126,3077044.0,40233.0,37001,37001126,126,2038,1713,520,921,32,...,30,1609,50,3,12,1,3,5,"POLYGON ((566985.479 260381.186, 567005.469 26...",37001126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3719901-BUR,64691125.0,399.0,37199,3719901-BUR,01 BUR,4409,3586,1304,1027,43,...,241,3245,66,6,5,0,1,22,"POLYGON ((311648.889 247675.511, 311674.554 24...",3719901-BUR
3719911-PRI,70000204.0,0.0,37199,3719911-PRI,11 PRI,1364,1082,364,427,21,...,26,1045,1,2,3,0,0,5,"POLYGON ((306609.771 244772.319, 306621.302 24...",3719911-PRI
3719910-PEN,106067845.0,0.0,37199,3719910-PEN,10 PEN,625,528,170,179,4,...,11,513,1,0,0,0,0,3,"POLYGON ((314145.914 228251.301, 313983.122 22...",3719910-PEN
